In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mord as m

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, Imputer, Normalizer, StandardScaler

# sklearn meteics for model performance
from sklearn.metrics import confusion_matrix,roc_curve, auc, roc_auc_score

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# for splitting data into train and test
from sklearn.cross_validation import train_test_split

import logging
logging.basicConfig(level=logging.DEBUG)

import statsmodels.api as sm
from patsy.highlevel import dmatrices

%matplotlib inline

DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown
DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


# Read Data

In [2]:
build_owner = pd.read_csv("Dataset/Building_Ownership_Use.csv")
build_struct = pd.read_csv("Dataset/Building_Structure.csv")
train = pd.read_csv("Dataset/train.csv")
test = pd.read_csv("Dataset/test.csv")


#### Details of the files are as follows: 

#### train.csv : 

|Varaible| Descrition|
|--------|-----------|
|area_assesed|Indicates the nature of the damage assessment in terms of the areas of the building that were assessed|
|building_id|A unique ID that identifies every individual building|
|damage_grade|Damage grade assigned to the building after assessment (Target Variable)|
|district_id|District where the building is located|
|has_geotechnical_risk|Indicates if building has geotechnical risks|
|has_geotechnical_risk_fault_crack|Indicates if building has geotechnical risks related to fault cracking|
|has_geotechnical_risk_flood|Indicates if building has geotechnical risks related to flood|
|has_geotechnical_risk_land_settlement|Indicates if building has geotechnical risks related to land settlement|
|has_geotechnical_risk_landslide|Indicates if building has geotechnical risks related to landslide|
|has_geotechnical_risk_liquefaction|Indicates if building has geotechnical risks related to liquefaction|
|has_geotechnical_risk_other|Indicates if building has any other  geotechnical risks|
|has_geotechnical_risk_rock_fall|Indicates if building has geotechnical risks related to rock fall|
|has_repair_started|Indicates if the repair work had started|
|vdcmun_id|Municipality where the building is located|

#### test.csv

Contains the same variables as the train.csv except the 'damage_grade' which is the target variable/ varaible to be predicted.

#### Building_Ownership_Use.csv: 

|Varaible|Description|
|--------|-----------|
|building_id|A unique ID that identifies every individual building|
|district_id|District where the building is located|
|vdcmun_id|Municipality where the building is located|
|ward_id|Ward Number in which the building is located|
|legal_ownership_status|Legal ownership status of the land in which the building was built|
|count_families|Number of families in the building|
|has_secondary_use|indicates if the building is used for any secondary purpose|
|has_secondary_use_agriculture|indicates if the building is secondarily used for agricultural purpose|
|has_secondary_use_hotel|indicates if the building is secondarily used as hotel|
|has_secondary_use_rental|indicates if the building is secondarily used for rental purpose|
|has_secondary_use_institution|indicates if the building is secondarily used for institutional purpose|
|has_secondary_use_school|indicates if the building is secondarily used as school|
|has_secondary_use_industry|indicates if the building is secondarily used for industrial purpose|
|has_secondary_use_health_post|indicates if the building is secondarily used as health post|
|has_secondary_use_gov_office|indicates if the building is secondarily used as government office|
|has_secondary_use_use_police|indicates if the building is secondarily used as police station|
|has_secondary_use_other|indicates if the building is secondarily used for other purpose|


#### Building_Structure.csv

|Variable|Description|
|--------|-----------|
|building_id|A unique ID that identifies every individual building|
|district_id|District where the building is located|
|vdcmun_id|Municipality where the building is located|
|ward_id|Ward Number in which the building is located|
|count_floors_pre_eq|Number of floors that the building had before the earthquake|
|count_floors_post_eq|Number of floors that the building had after the earthquake|
|age_building|Age of the building (in years)|
|plinth_area_sq_ft|Plinth area of the building (in square feet)|
|height_ft_pre_eq|Height of the building before the earthquake (in feet)|
|height_ft_post_eq|Height of the building after the earthquake (in feet)|
|land_surface_condition|Surface condition of the land in which the building is built	|
|foundation_type|Type of foundation used in the building|
|roof_type|Type of roof used in the building|
|ground_floor_type|Ground floor type|
|other_floor_type|Type of construction used in other floors (except ground floor and roof)|
|position|Position of the building|
|plan_configuration|Building plan configuration|
|has_superstructure_adobe_mud|indicates if the superstructure of the building is made of Adobe/Mud|
|has_superstructure_mud_mortar_stone|indicates if the superstructure of the building is made of Mud Mortar - Stone|
|has_superstructure_stone_flag| indicates if the superstructure of the building is made of Stone|
|has_superstructure_mud_mortar_brick|indicates if the superstructure of the building is made of Cement Mortar - Stone|
|has_superstructure_cement_mortar_brick|indicates if the superstructure of the building is made of Mud Mortar - Brick|
|has_superstructure_timber|indicates if the superstructure of the building is made of Timber|
|has_superstructure_bamboo|indicates if the superstructure of the building is made of Bamboo|
|has_superstructure_rc_non_engineered|indicates if the superstructure of the building is made of RC (Non Engineered)|
|has_superstructure_rc_engineered|indicates if the superstructure of the building is made of RC (Engineered)|
|has_superstructure_other| indicates if the superstructure of the building is made of any other material|
|condition_post_eq|Actual contition of the building after the earthquake|


In [3]:
len(train) + len(test), len(build_owner), len(build_struct)

(1052936, 1052948, 1052948)

In [4]:
data = (train.merge(build_struct, how='inner', on=['building_id','district_id','vdcmun_id']))
data = (data.merge(build_owner, how='inner', on=['building_id','district_id','vdcmun_id','ward_id']))

In [5]:
data.head()

,area_assesed,building_id,damage_grade,district_id,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,Both,24385bfd2a2,Grade 4,24,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Both,405d1bbebbf,Grade 2,44,0.0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,Both,351d9bc71f6,Grade 1,36,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Building removed,2be3a971166,Grade 5,30,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Both,34c7d073ea6,Grade 3,36,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Exploratory Data Analysis

## Column Values


Let's look at the number of columns of each data type. **int64** and **float64** are numeric variables (which can be either discrete or continuous). **object** columns contain strings and are categorical features.

In [6]:
print(f'dtype value counts:\n{data.dtypes.value_counts()}')
data_types = pd.concat([data.dtypes, data.apply(pd.Series.nunique)], axis=1)
data_types = data_types.rename(columns = {0: "dtype", 1: "#unique values"})
data_types.sort_values(['#unique values'])

dtype value counts:
int64      37
object     12
float64     4
dtype: int64


,dtype,#unique values
has_secondary_use_other,int64,2
has_superstructure_cement_mortar_stone,int64,2
has_superstructure_stone_flag,int64,2
has_superstructure_mud_mortar_stone,int64,2
has_superstructure_adobe_mud,int64,2
has_superstructure_timber,int64,2
has_secondary_use_use_police,int64,2
has_superstructure_bamboo,int64,2
has_superstructure_rc_non_engineered,int64,2
has_superstructure_rc_engineered,int64,2


## Change data types to correct data types

In [7]:
data['ward_id'] = data['ward_id'].astype(object)
data['vdcmun_id'] = data['vdcmun_id'].astype(object)
data['district_id'] = data['district_id'].astype(object)

## Examine Missing Values

In [8]:
def missing_values_table(data):
    # Total missing values
    mis_val = data.isnull().sum()

    # Percent of missing values
    mis_val_percent = 100 * data.isnull().sum()/len(data)

    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis = 1)

    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0: "Missing Values", 1: "% of Total Values"})

    # Concatenate the datatype information
    data_types = pd.concat([data.dtypes, data.apply(pd.Series.nunique)], axis=1)
    data_types = data_types.rename(columns = {0: "dtype", 1: "#unique values"})
    mis_val_table_ren_columns = pd.concat([data_types, mis_val_table_ren_columns], axis=1)
    
    # Sort the table by percent of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns.sort_values("% of Total Values", ascending = False).round(1)
         
    
    # Print some summary information
    print ("Your selected dataframe has " + str(data.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,3] != 0].shape[0]) +
            " columns that have missing values.")
    
    return mis_val_table_ren_columns

In [9]:
# Missing values statistics
missing_values = missing_values_table(data)
missing_values

Your selected dataframe has 53 columns.
There are 1 columns that have missing values.


,dtype,#unique values,Missing Values,% of Total Values
has_repair_started,float64,2,33417,5.3
count_families,float64,11,1,0.0
legal_ownership_status,object,4,0,0.0
has_superstructure_mud_mortar_stone,int64,2,0,0.0
has_superstructure_stone_flag,int64,2,0,0.0
has_superstructure_cement_mortar_stone,int64,2,0,0.0
has_superstructure_mud_mortar_brick,int64,2,0,0.0
has_superstructure_cement_mortar_brick,int64,2,0,0.0
has_superstructure_timber,int64,2,0,0.0
has_superstructure_bamboo,int64,2,0,0.0


## Examine the distribution of target column

In [10]:
# Fraction of signup drivers who completed the first trip
target_dist = pd.concat([data['damage_grade'].value_counts(), 100 * data['damage_grade'].value_counts()/len(data)], axis=1)
target_dist.columns = ['target_count','target_fraction (in %)']
target_dist

,target_count,target_fraction (in %)
Grade 5,210825,33.371006
Grade 4,152244,24.098354
Grade 3,122288,19.356687
Grade 2,85084,13.467751
Grade 1,61320,9.706202


# Feature Engineering

In [11]:
data['change_floors_post_eq'] = data['count_floors_post_eq'] - data['count_floors_pre_eq']

In [12]:
data['change_height_ft_post_eq'] = data['height_ft_post_eq'] - data['height_ft_pre_eq']

In [13]:
ori_data = data.copy(deep=True)

In [33]:
data = ori_data.copy(deep=False)

In [34]:
data = data.drop(columns=['building_id','ward_id','vdcmun_id'])

# Imputing Missing Values

In [35]:
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [36]:
data = DataFrameImputer().fit_transform(data)
missing_values_table(data)

Your selected dataframe has 52 columns.
There are 0 columns that have missing values.


,dtype,#unique values,Missing Values,% of Total Values
area_assesed,object,5,0,0.0
damage_grade,object,5,0,0.0
has_superstructure_cement_mortar_stone,int64,2,0,0.0
has_superstructure_mud_mortar_brick,int64,2,0,0.0
has_superstructure_cement_mortar_brick,int64,2,0,0.0
has_superstructure_timber,int64,2,0,0.0
has_superstructure_bamboo,int64,2,0,0.0
has_superstructure_rc_non_engineered,int64,2,0,0.0
has_superstructure_rc_engineered,int64,2,0,0.0
has_superstructure_other,int64,2,0,0.0


# Encoding Categorical Variables

In [37]:
# Create a label encoder object
le = LabelEncoder()
le_count = 0

for col in data:
    if data[col].dtype == 'object' or col == 'damage_grade':
        # if 2 or fewer unique categories
        if(data[col].nunique() <= 2 or col == 'damage_grade'):
            # Train on training data
            le.fit(data[col])
            # Transform both training and test data
            data[col] = le.transform(data[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1

print('%d columns were label encoded.' % le_count)

1 columns were label encoded.


In [38]:
train_y = data['damage_grade'] + 1

In [39]:
# one-hot-encoding of categorical variables
train_X = pd.get_dummies(data.loc[:,data.columns != 'damage_grade'])

print('Training Feature Size: ', train_X.shape)

Training Feature Size:  (631761, 92)


# Baseline

## Decision Tree

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = DecisionTreeClassifier(random_state=0)

In [ ]:
cross_val_score(clf, train_X, train_y, cv=10)

In [ ]:
clf_1 = RandomForestClassifier(random_state=0,n_estimators = 100)

In [ ]:
#cross_val_score(clf_1, train_X, train_y, cv=10)

In [ ]:
clf_2 = m.LogisticIT() #Default parameters: alpha=1.0, verbose=0, maxiter=10000

In [40]:
test_data = (test.merge(build_struct, how='inner', on=['building_id','district_id','vdcmun_id']))
test_data = (test_data.merge(build_owner, how='inner', on=['building_id','district_id','vdcmun_id','ward_id']))

test_data['ward_id'] = test_data['ward_id'].astype(object)
test_data['vdcmun_id'] = test_data['vdcmun_id'].astype(object)
test_data['district_id'] = test_data['district_id'].astype(object)
missing_values = missing_values_table(test_data)
#missing_values

Your selected dataframe has 52 columns.
There are 1 columns that have missing values.


In [41]:
test_data['change_floors_post_eq'] = test_data['count_floors_post_eq'] - test_data['count_floors_pre_eq']
test_data['change_height_ft_post_eq'] = test_data['height_ft_post_eq'] - test_data['height_ft_pre_eq']
test_data = test_data.drop(columns=['building_id','ward_id','vdcmun_id'])
test_data = DataFrameImputer().fit_transform(test_data)

In [42]:
test_data = pd.get_dummies(test_data.loc[:,test_data.columns != 'damage_grade'])
print('Training Feature Size: ', test_data.shape)

Training Feature Size:  (421175, 92)


In [24]:
scaler = StandardScaler()

# Repeat with the scaler
train_X = scaler.fit_transform(train_X)
test_data = scaler.transform(test_data)

In [43]:
import xgboost as xgb # XGBoost implementation

In [ ]:
gbm = xgb.XGBClassifier(max_depth=6, n_estimators=150, learning_rate=0.5, objective= "multi:softprob", silent=False)
gbm.fit(train_X, train_y, eval_metric='mlogloss')

In [ ]:
pred = gbm.predict(test_data)

In [ ]:
submission = pd.DataFrame({ 'building_id': test.building_id,
                            'damage_grade': pred })

submission['damage_grade'] = submission['damage_grade'].astype(str)
submission['damage_grade'] = "Grade " + submission.loc[:,'damage_grade']

In [32]:
submission.to_csv('Dataset/submission_8.csv', index=False)

In [ ]:
clf_1.fit(X=train_X,y=train_y)

In [ ]:
pred = clf_1.predict(test_data)

In [ ]:
prediction = pd.DataFrame(pred)
prediction['building_id'] = test.building_id
prediction.columns = ['damage_grade','building_id']
prediction = prediction[['building_id', 'damage_grade']]
prediction['damage_grade'] = prediction['damage_grade'].astype(str)
prediction['damage_grade'] = "Grade " + prediction.loc[:,'damage_grade']

In [ ]:
prediction.to_csv('Dataset/submission_5.csv', index=False)